In [4]:
from gdeltdoc import GdeltDoc, Filters
import pandas as pd
import datetime
from tqdm import tqdm
import dataframe_image as dfi
import time

In [ ]:
f = Filters(
    keyword = ["Union of Soviet Socialist Republics", "USSR"],
    start_date = "2017-04-10",
    end_date = "2022-05-11",
    num_records = 250,
    domain = "rg.ru",
    country = "RS"
)

gd = GdeltDoc()

# Search for articles matching the filters
articles = gd.article_search(f)

# Get a timeline of the number of articles matching the filters
timeline = gd.timeline_search("timelinevol", f)

In [ ]:
def get_gdelt_timeline(keyword, start_date, end_date, domain, country):
    f = Filters(
        keyword=keyword,
        start_date=start_date,
        end_date=end_date,
        num_records=250,
        domain=domain,
        country=country,
    )

    gd = GdeltDoc()

    timeline = gd.timeline_search("timelinevol", f)

    return timeline


In [ ]:
timeline = get_gdelt_timeline(
    keyword=["Union of Soviet Socialist Republics", "USSR"],
    start_date="2017-01-01",
    end_date="2018-01-01",
    domain="rg.ru",
    country="RS",
)

timeline.to_csv("timeline.csv")

In [ ]:
DeepnoteChart(timeline, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"trail","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":"ascending","type":"temporal","field":"datetime","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"Volume Intensity","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":"ascending","type":"temporal","field":"datetime","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"Volume Intensity","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [5]:
def get_gdelt_articles(keyword, start_date, end_date, domain, country):
    articles = pd.DataFrame()

    start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d").date()
    end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d").date()

    delta = datetime.timedelta(days=1)
    start_delta_date = start_date
    end_delta_date = start_date + delta

    n_days = int((end_date - start_date).days)
    pbar = tqdm(desc="Progress", total=n_days, leave=False)

    while end_delta_date <= end_date:
        f = Filters(
            keyword=keyword,
            start_date=start_delta_date.strftime("%Y-%m-%d"),
            end_date=end_delta_date.strftime("%Y-%m-%d"),
            num_records=250,
            domain=domain,
            country=country,
        )

        gd = GdeltDoc()

        articles = pd.concat([articles, gd.article_search(f)])

        # print(len(articles))

        start_delta_date += delta
        end_delta_date += delta

        pbar.update(1)
        time.sleep(1)
    return articles


In [6]:
articles = get_gdelt_articles(
    keyword=["Union of Soviet Socialist Republics", "USSR"],
    start_date="2022-01-01",
    end_date="2023-01-01",
    domain="rg.ru",
    country="RS",
)
articles.to_csv("articles.csv")
articles

,url,url_mobile,title,seendate,socialimage,domain,language,sourcecountry
0,https://rg.ru/2022/01/01/reg-sibfo/altaec-sorv...,https://rg.ru/amp/2022/01/01/reg-sibfo/altaec-...,Алтаец сорвался со скалы в Катунь во время зап...,20220101T151500Z,,rg.ru,Russian,Russia
1,https://rg.ru/2022/01/01/on-razrushal-plastiko...,https://rg.ru/amp/2022/01/01/on-razrushal-plas...,Он разрушал пластиковую красоту балетной услов...,20220101T094500Z,https://cdnimg.rg.ru/img/content/223/85/36/000...,rg.ru,Russian,Russia
2,https://rg.ru/2022/01/01/pryzhok-ivana-groznog...,https://rg.ru/amp/2022/01/01/pryzhok-ivana-gro...,Прыжок Ивана Грозного : Как Юрий Владимиров из...,20220101T134500Z,https://cdnimg.rg.ru/img/content/223/87/05/655...,rg.ru,Russian,Russia
0,https://rg.ru/2022/01/02/putin-nazval-grigorov...,https://rg.ru/amp/2022/01/02/putin-nazval-grig...,Путин назвал Григоровича патриархом российской...,20220102T111500Z,,rg.ru,Russian,Russia
1,https://rg.ru/2022/01/02/mishustin-pozdravil-g...,https://rg.ru/amp/2022/01/02/mishustin-pozdrav...,Мишустин поздравил Григоровича с 95 - летием,20220102T103000Z,,rg.ru,Russian,Russia
...,...,...,...,...,...,...,...,...
14,https://rg.ru/2022/12/30/odin-iz-samyh-poseshc...,https://rg.ru/amp/2022/12/30/odin-iz-samyh-pos...,Один из самых посещаемых музеев СССР Горки Ле...,20221230T120000Z,https://cdnstatic.rg.ru/uploads/images/2022/12...,rg.ru,Russian,Russia
15,https://rg.ru/2022/12/30/aleksandr-kliuchko-da...,https://rg.ru/amp/2022/12/30/aleksandr-kliuchk...,"Концерты недели : Ключко , Никифорчин , Поспел...",20221230T133000Z,https://cdnstatic.rg.ru/uploads/images/2022/12...,rg.ru,Russian,Russia
16,https://rg.ru/2022/12/30/gospodderzhka-apk-v-2...,https://rg.ru/amp/2022/12/30/gospodderzhka-apk...,Господдержка АПК в 2023 году будет расширена -...,20221230T120000Z,https://cdnstatic.rg.ru/uploads/images/2022/12...,rg.ru,Russian,Russia
17,https://rg.ru/2022/12/30/provereno-na-sebe-kak...,https://rg.ru/amp/2022/12/30/provereno-na-sebe...,Проверено на себе : Как раньше в школах и вуза...,20221230T120000Z,,rg.ru,Russian,Russia


In [ ]:
articles_viz = pd.read_csv("/work/rg/2019/articles.csv")[['url', 'title', 'seendate', 'domain', 'language']][:5]
articles_viz['url'] = articles_viz['url'].str.slice(0,27) + "..."
articles_viz['title'] = articles_viz['title'].str.slice(0,47) + "..."
dfi.export(articles_viz.style, "articles.png", table_conversion="matplotlib")
articles_viz.style

,url,title,seendate,domain,language
0,https://rg.ru/2019/01/01/no...,Новый год начинается с юбилея великого писателя...,20190101T111500Z,rg.ru,Russian
1,https://rg.ru/2019/01/01/po...,Порошенко возвел в ранг официального праздника ...,20190101T094500Z,rg.ru,Russian
2,https://rg.ru/2019/01/01/ba...,Басинский : Гранин и Солженицын оказались литер...,20190101T061500Z,rg.ru,Russian
3,https://rg.ru/2019/01/02/pu...,Путин поздравил хореографа Григоровича с днем р...,20190102T110000Z,rg.ru,Russian
4,https://rg.ru/2019/01/02/dm...,Дмитрий Медведев поздравил с юбилеем обозревате...,20190102T121500Z,rg.ru,Russian


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0d79deab-c9c6-414b-8803-1e385542ff1f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>